In [1]:
# %pip install pandas

In [2]:
import pandas as pd

below code snippet is taken from: <br> https://gist.github.com/mridul1012/9867d3b5dc77e0f12d5568b47b7ff9a2#file-m2_to_csv-py
  https://www.cl.cam.ac.uk/research/nl/bea2019st/data/corr_from_m2.py

In [3]:
def process_m2file(m2_file, out_file, id=0):

  """
  This function used to convert .m2 file to .txt file. 
  This functions return dataframe with correct, incorrect, total_edits 

  Arguments:
  m2_file: path to .m2 file
  out: path to .txt file
  id: Annotator id

  converts specified annotators text from erroneous to non-erroneous
  """
  
  m2 = open(m2_file).read().strip().split("\n\n")
  out = open(out_file, "w")
  # Do not apply edits with these error types
  skip = {"noop", "UNK", "Um"}
  total_edits = []
  
  for sent in m2:
    sent = sent.split("\n")
    cor_sent = sent[0].split()[1:] # Ignore "S "
    edits = sent[1:]
    offset = 0
    cur_edit = 0
    for edit in edits:
      edit = edit.split("|||")
      if edit[1] in skip: continue # Ignore certain edits
      coder = int(edit[-1])
      if coder != id: continue # Ignore other coders
      span = edit[0].split()[1:] # Ignore "A "
      start = int(span[0])
      end = int(span[1])
      cor = edit[2].split()
      cor_sent[start+offset:end+offset] = cor
      cur_edit+=1
      offset = offset-(end-start)+len(cor)
    total_edits.append(cur_edit)
    out.write(" ".join(cor_sent)+"\n")
  out.close()

  file1 = open(m2_file,"r")
  s1 = file1.read()

  each_sent = s1.split("\n\n")

  incorrect = []
  for i in range(len(each_sent)):
      temp = each_sent[i].split("\n")
      temp = temp[0]
      temp = temp.split(" ")
      temp = temp[1:]# ignore S
      temp = ' '.join(temp)
      incorrect.append(temp)
      
  #preprocessing correct sentences

  file2 = open(out_file, "r")
  s2 = file2.read()

  correct = s2.split("\n")

  print("Preparing dataframe")
  df = pd.DataFrame()
  df["correct"] = correct
  df["incorrect"] = incorrect
  total_edits.append(0)
  df['total_edits'] = total_edits

  return df

## Extracting data from Annatator 0

In [4]:
data_id0 = process_m2file(m2_file='/home/seebase/CS4248/Project/ABC.train.gold.bea19.300samples.m2',
                      out_file='/home/seebase/CS4248/Project/ABC_train_gold_bea19_300samples_corrected_sentences.txt',
                      id=0)

Preparing dataframe


In [5]:
data_id0.shape

(301, 3)

In [6]:
#save dataframe
data_id0.to_csv('/home/seebase/CS4248/Project/ABC_train_gold_bea19_300samples.csv', index=False)